# Load in dependencies

In [1]:
#ensure that matplotlib plots inline
%matplotlib inline

import os
import numpy as np
import mvpa2
import nibabel
import pickle
import gzip
from mvpa2.suite import *
from nilearn.image import resample_img
from nilearn import image
from scipy.signal import resample

E:\Anaconda3\lib\site-packages\mvpa2\base\dochelpers.py:167: FutureWarning: split() requires a non-empty pattern match.
  entries = __re_spliter1.split(paramdoc)
E:\Anaconda3\lib\site-packages\mvpa2\base\dochelpers.py:167: FutureWarning: split() requires a non-empty pattern match.
  entries = __re_spliter1.split(paramdoc)
E:\Anaconda3\lib\site-packages\scipy\_lib\decorator.py:205: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  first = inspect.getargspec(caller)[0][0]  # first arg
E:\Anaconda3\lib\site-packages\scipy\optimize\nonlin.py:1502: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, varkw, defaults = inspect.getargspec(jac.__init__)
E:\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:636: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  sign = inspect.getargspec(self._stats)
E:\Anaconda3\lib\site-packages\scipy\stats\_distn_in

 * Please note: warnings are printed only once, but underlying problem might occur many times *


E:\Anaconda3\lib\site-packages\mvpa2\base\dochelpers.py:167: FutureWarning: split() requires a non-empty pattern match.
  entries = __re_spliter1.split(paramdoc)
E:\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:
E:\Anaconda3\lib\site-packages\scipy\_lib\decorator.py:205: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  first = inspect.getargspec(caller)[0][0]  # first arg
E:\Anaconda3\lib\site-packages\mvpa2\base\dochelpers.py:167: FutureWarning: split() requires a non-empty pattern match.
  entries = __re_spliter1.split(paramdoc)
E:\Anaconda3\lib\site-packages\mvpa2\datasets\sources\skl_data.py:32: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  argnames, varargs, varkw, defaults = inspect.getargspec(fx)
E:\Anaconda3\lib\site-packages\pandas\__init__.py:7: Depre

## Load in the data set

In [2]:
# get the location of the files
bold_fname = os.path.join('..', 'data', 'qub', '4Dw.nii')
mask_fname = os.path.join('..', 'data', 'qub', 'struct', 'final_mask_w.hdr')

In [3]:
# create a pymvpa fmri dataset object
original_dataset = fmri_dataset(bold_fname, mask=mask_fname)

## Now we will load in our trajectory

In [4]:
#get the trajectory
with open('..\\data\\story\\smoothedTrajectoriesDifferentWindowSizes\\smoothTrajRawValenceMean_win200.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    original_trajectory = u.load()

In [5]:
#downsample the trajectory to fit match the number of fMRI sample
resampled_trajectory = resample(original_trajectory, len(original_dataset.samples))

In [6]:
original_dataset.shape

(1351, 65159)

In [7]:
original_dataset.sa['trajectory'] = resampled_trajectory

In [8]:
temp_array = []

#add a sample runtype
for i in range(len(original_dataset)):
    if i % 2 != 0:
        temp_array.append('odd')
    else:
        temp_array.append('even')

In [9]:
original_dataset.sa['runtype'] = temp_array

# Classification time

In [10]:
# set up the classifier
classifier = kNN(k=3, dfx=one_minus_correlation, voting='majority')

In [11]:
original_dataset.sa['targets'] = resampled_trajectory

In [12]:
# train the dataset
classifier.train(original_dataset)

In [13]:
original_dataset.samples

memmap([[31, 28, 30, ..., 10, 10, 11],
       [32, 28, 29, ..., 11, 11, 11],
       [32, 29, 30, ..., 11, 11, 11],
       ..., 
       [30, 29, 32, ..., 10, 11, 11],
       [30, 29, 33, ..., 10, 11, 11],
       [30, 29, 33, ..., 11, 11, 11]], dtype=int16)

In [17]:
predictions = clf.predict(original_dataset.samples)

In [18]:
np.mean(predictions == original_dataset.sa.targets)

0.0

In [14]:
clf = LinearCSVMC()
cvte = CrossValidation(clf, HalfPartitioner(attr='runtype'))

In [15]:
cv_results = cvte(original_dataset)

E:\Anaconda3\lib\site-packages\mvpa2\generators\partition.py:294: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return [(None, uniqueattrs[:len(uniqueattrs)/2]),
E:\Anaconda3\lib\site-packages\mvpa2\generators\partition.py:295: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  (None, uniqueattrs[len(uniqueattrs)/2:])]


In [16]:
np.mean(cv_results)

1.0

# Split up data for test speed

In [22]:
%time
ds_split1 = original_dataset[original_dataset.sa.runtype == 'odd']
len(ds_split1)

Wall time: 0 ns


675

In [23]:
ds_split2 = original_dataset[original_dataset.sa.runtype == 'even']
len(ds_split2)

676

In [24]:
clf = kNN(k=1, dfx=one_minus_correlation, voting='majority')

In [25]:
clf.set_postproc(BinaryFxNode(mean_mismatch_error, 'targets'))

In [26]:
clf.train(ds_split2)

In [27]:
%time
err = clf(ds_split1)

Wall time: 0 ns


In [29]:
%time
np.asscalar(err)

Wall time: 0 ns


1.0

In [33]:
%time
predictions = clf.predict(ds_split1)

Wall time: 0 ns


In [35]:
np.mean(predictions == ds_split1.sa.targets)

0.0

In [38]:
mccourt = 10 + 11

In [41]:
new_classifier = kNN(k=1, dfx=one_minus_correlation, voting='majority')

In [42]:
cvte = CrossValidation(new_classifier, HalfPartitioner(attr='runtype'))

In [43]:
cv_results = cvte(original_dataset)

E:\Anaconda3\lib\site-packages\mvpa2\generators\partition.py:294: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return [(None, uniqueattrs[:len(uniqueattrs)/2]),
E:\Anaconda3\lib\site-packages\mvpa2\generators\partition.py:295: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  (None, uniqueattrs[len(uniqueattrs)/2:])]


In [44]:
np.mean(cv_results)

1.0

# trying to get classification working :/

In [169]:
#create a really small array to work with
small_array = []

#size
size = int(len(original_dataset)/8)

print(size)

#new size
new_ds = original_dataset[size*-1 : ]

168


In [170]:
new_ds.shape

(168, 65159)

In [171]:
small_classifier = kNN(k=1, dfx=one_minus_correlation, voting='majority')

In [172]:
small_classifier.train(new_ds)

In [173]:
#predict against itself
small_predictions = small_classifier.predict(new_ds.samples)

In [174]:
np.mean(small_predictions == new_ds.sa.targets)

1.0

In [175]:
small_cvte = CrossValidation(small_classifier, HalfPartitioner(attr='runtype'))

In [176]:
small_cv_results = small_cvte(new_ds)

E:\Anaconda3\lib\site-packages\mvpa2\generators\partition.py:294: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return [(None, uniqueattrs[:len(uniqueattrs)/2]),
E:\Anaconda3\lib\site-packages\mvpa2\generators\partition.py:295: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  (None, uniqueattrs[len(uniqueattrs)/2:])]


In [177]:
np.mean(small_cv_results)

1.0

In [178]:
#other_new_ds = original_dataset[ : size]
other_new_ds = copy.copy(new_ds)
other_new_ds.shape

(168, 65159)

In [179]:
other_new_ds.targets

array([  1.49278800e+00,   1.56543925e+00,   1.50255015e+00,
         2.04295041e+00,   3.86883085e+00,   3.77315104e+00,
         4.00653734e+00,   3.64025896e+00,   3.50984472e+00,
         3.66923133e+00,   2.22048909e+00,   1.81657309e+00,
         2.41480677e+00,  -3.90234246e-01,  -5.41969051e-01,
        -6.00965334e-01,  -1.76439973e-01,  -9.18920323e-02,
         3.10778943e-01,   1.11577768e+00,   9.84884622e-01,
         1.08122708e+00,   1.05086404e+00,   1.10067884e+00,
         1.07505916e+00,   2.08172420e-01,   4.19552406e-02,
         8.82787961e-02,   4.27503765e-02,   8.39139029e-02,
         8.93544125e-02,   5.04932451e-01,   9.30582229e-01,
         1.03033737e+00,   1.06718055e+00,   1.14874236e+00,
         1.17871938e+00,   3.60853416e-01,   2.28762579e-01,
         3.45107967e-01,   3.28331749e-01,   4.10559608e-01,
         4.02443981e-01,   4.77271266e-01,   4.61712240e-01,
         5.46412284e-01,   5.27012310e-01,   6.11084917e-01,
         5.77172090e-01,

In [180]:
#other_new_ds.sa.pop("targets", None)
print("done")

done


In [188]:
other_small_predictions = small_classifier.predict(other_new_ds.samples)

In [186]:
np.mean(other_small_predictions == new_ds.sa.targets)

0.5

In [187]:
other_small_predictions

[0.083913902908338334,
 1.5654392493517786,
 1.5654392493517786,
 2.0429504115901125,
 -0.60096533379764849,
 3.773151035407007,
 -2.1546163269427874,
 3.6402589589531207,
 -5.8638582455548471,
 3.6692313341402065,
 3.6692313341402065,
 1.8165730880184063,
 1.8165730880184063,
 -0.39023424628288811,
 3.6692313341402065,
 -0.60096533379764849,
 3.6692313341402065,
 -0.091892032258578851,
 -0.39023424628288811,
 1.1157776841989413,
 -1.6078612020585472,
 1.0812270752309932,
 1.0812270752309932,
 1.1006788445598314,
 1.1006788445598314,
 0.20817242039148687,
 0.4105596082084077,
 0.08827879607553836,
 -2.1546163269427874,
 0.083913902908338334,
 0.50493245148802779,
 0.50493245148802779,
 0.50493245148802779,
 1.030337368108077,
 1.1487423640913637,
 1.1487423640913637,
 0.36085341626117157,
 0.36085341626117157,
 0.36085341626117157,
 0.34510796665374815,
 0.4105596082084077,
 0.4105596082084077,
 0.4105596082084077,
 0.47727126562812228,
 -2.1546163269427874,
 0.54641228438239664,
 -1.6

In [5]:
#get the trajectory
with open('..\\data\\sample\\results.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    traj = u.load()

In [6]:
for value in traj:
    print(value)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
-1
-1
1
1
1
-1
-1
1
1
-1
-1
1
1
1
1
-1
-1
-1
-1
-1
1
1
1
1
1
1
-1
1
-1
1
1
-1
-1
1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
1
1
1
1
-1
-1
1
-1
-1
-1
1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
